In [2]:
!pip install -r /content/drive/MyDrive/5th_sem/Ee603_mlsp/Project/requirements.txt

     |████████████████████████████████| 2.1 MB 5.0 MB/s 
  Created wheel for dcase-util: filename=dcase_util-0.2.18-py3-none-any.whl size=2147231 sha256=8f5a85792dde29774a34105f0c16ced5084b95bd1544eac581dc0ee5bdb1b0ef
  Stored in directory: /root/.cache/pip/wheels/f2/75/d1/e4fc6f415d1100dc30c39c955aa2bfbd0d88c6138076e74793
  Created wheel for sed-eval: filename=sed_eval-0.2.1-py3-none-any.whl size=26124 sha256=353e0ee75bbc7767cf01e216dd8d66cdaa63c9892e22ade57e6e10f2f994d3b6
  Stored in directory: /root/.cache/pip/wheels/96/19/00/a3ec89acfdbd2568185a1ae3020590e76ee73a47ac4ff83e84
Successfully built dcase-util sed-eval


In [3]:
import tensorflow.compat.v1 as tf
#tf.disable_v2_behavior()

In [4]:
import numpy as np
import os
import librosa
import librosa.display

In [5]:
! pip install dcase_util
import dcase_util

In [7]:
import dcase_util
from dcase_util.containers import AudioContainer
import os, numpy
from tqdm import tqdm_notebook as tqdm
import matplotlib.pyplot as plt
import sed_eval
%matplotlib inline

# Handy tool to print data in HTML form
log = dcase_util.ui.FancyHTMLPrinter()

# Paths to store data
data_storage_path = '/content/data'
dataset_storage_path = os.path.join(data_storage_path, 'datasets')
feature_storage_path = os.path.join(data_storage_path, 'features_sed')
dcase_util.utils.Path().create(
    [data_storage_path, dataset_storage_path, feature_storage_path]
)

# Filename for acoustic model
model_filename = os.path.join(data_storage_path, 'model_sed.h5')  
    
# Setup Keras to use tensorflow as backend
dcase_util.keras.setup_keras(backend='tensorflow')
import keras
from keras.layers import *
from keras.models import Model

In [12]:
def csv_to_meta_container(csv_file_path):
  dict_container = dcase_util.containers.ListDictContainer(filename = csv_file_path)
  dict_container.load()
  train_meta = dcase_util.containers.MetaDataContainer(dict_container)
  print(train_meta)
  return train_meta

In [84]:
train_meta = csv_to_meta_container('/content/hmm_mock_1.csv')

MetaDataContainer :: Class
Items                               : 178 
Unique
  Files                             : 10 
  Scene labels                      : 0 
  Event labels                      : 2 
  Tags                              : 0 
  Identifiers                       : 0 
  Source labels                     : 0 

Meta data
  Source                  Onset   Offset   Scene             Event             Tags              Identifier   
  --------------------   ------   ------   ---------------   ---------------   ---------------   -----   
  test_sample-9.npy        0.00     2.04   -                 speech            -                 -       
  test_sample-9.npy        2.04     2.08   -                 music             -                 -       
  test_sample-9.npy        2.08     2.56   -                 speech            -                 -       
  test_sample-9.npy        2.56     2.59   -                 music             -                 -       
  test_sample-9.npy     

In [85]:
res = dcase_util.containers.MetaDataContainer(filename=os.path.join(data_storage_path, 'results_sed_hmm.csv'))
for file_name in train_meta.unique_files:
  event_list = train_meta.filter(filename=file_name)
  event_list = event_list.process_events(minimum_event_gap = 0.5)
  event_list = event_list.process_events(minimum_event_length = 0.7)
  print(file_name)
  res += event_list

res = res.filter(event_list = ['speech', 'music'])

res.save('results_hmm_mock_1.csv')
# res.save('temp.csv')

test_sample-0.npy
test_sample-1.npy
test_sample-2.npy
test_sample-3.npy
test_sample-4.npy
test_sample-5.npy
test_sample-6.npy
test_sample-7.npy
test_sample-8.npy
test_sample-9.npy


[{'event_label': 'music',
  'filename': 'test_sample-0.npy',
  'offset': 9.392971246,
  'onset': 7.31629393},
 {'event_label': 'speech',
  'filename': 'test_sample-0.npy',
  'offset': 10.0,
  'onset': 0.0},
 {'event_label': 'speech',
  'filename': 'test_sample-1.npy',
  'offset': 9.073482428,
  'onset': 0.0},
 {'event_label': 'music',
  'filename': 'test_sample-2.npy',
  'offset': 3.961661342,
  'onset': 1.916932907},
 {'event_label': 'music',
  'filename': 'test_sample-2.npy',
  'offset': 6.42172524,
  'onset': 4.984025559},
 {'event_label': 'speech',
  'filename': 'test_sample-2.npy',
  'offset': 2.300319489,
  'onset': 0.0},
 {'event_label': 'speech',
  'filename': 'test_sample-2.npy',
  'offset': 10.0,
  'onset': 3.833865815},
 {'event_label': 'speech',
  'filename': 'test_sample-3.npy',
  'offset': 10.0,
  'onset': 0.0},
 {'event_label': 'music',
  'filename': 'test_sample-4.npy',
  'offset': 2.587859425,
  'onset': 1.469648562},
 {'event_label': 'speech',
  'filename': 'test_samp

In [86]:
# print(len(res), len(train_meta))

In [87]:
# res = res.filter(event_list = ['speech', 'music'])
# print(res)

In [88]:
# type(res)

In [89]:
# type(train_meta)

In [77]:
# res.save('temp.csv')

In [78]:
############# Evaluating #####################

## Preparing data for evaluation

Prepare reference data and estimated to have filenames in uniform format:

In [79]:
validation_data = csv_to_meta_container('/content/True_Labels_Task_1.csv')

MetaDataContainer :: Class
Items                               : 14 
Unique
  Files                             : 10 
  Scene labels                      : 0 
  Event labels                      : 2 
  Tags                              : 0 
  Identifiers                       : 0 
  Source labels                     : 0 

Meta data
  Source                  Onset   Offset   Scene             Event             Tags              Identifier   
  --------------------   ------   ------   ---------------   ---------------   ---------------   -----   
  test_sample-0            6.23     9.95   -                 Music             -                 -       
  test_sample-1            2.30     6.40   -                 Speech            -                 -       
  test_sample-1            7.10     9.80   -                 Music             -                 -       
  test_sample-2            2.30     3.98   -                 Music             -                 -       
  test_sample-3          

In [80]:
reference_event_list = validation_data
estimated_event_list = dcase_util.containers.MetaDataContainer(
    filename=os.path.join('/content/', 'results_task1_spectro.csv')
).load()

for item_id, item in enumerate(estimated_event_list):
    estimated_event_list[item_id]['filename'] = item.filename.split('.')[0]    

In [81]:
print(estimated_event_list)

MetaDataContainer :: Class
Filename                            : /content/results_task1_spectro.csv 
Items                               : 28 
Unique
  Files                             : 10 
  Scene labels                      : 0 
  Event labels                      : 2 
  Tags                              : 0 
  Identifiers                       : 0 
  Source labels                     : 0 

Meta data
  Source                  Onset   Offset   Scene             Event             Tags              Identifier   
  --------------------   ------   ------   ---------------   ---------------   ---------------   -----   
  test_sample-0            1.82     5.08   -                 music             -                 -       
  test_sample-0            1.82     2.52   -                 speech            -                 -       
  test_sample-0            5.05    10.00   -                 speech            -                 -       
  test_sample-1            1.63     5.02   -             

In [82]:
print(reference_event_list)

MetaDataContainer :: Class
Items                               : 14 
Unique
  Files                             : 10 
  Scene labels                      : 0 
  Event labels                      : 2 
  Tags                              : 0 
  Identifiers                       : 0 
  Source labels                     : 0 

Meta data
  Source                  Onset   Offset   Scene             Event             Tags              Identifier   
  --------------------   ------   ------   ---------------   ---------------   ---------------   -----   
  test_sample-0            6.23     9.95   -                 Music             -                 -       
  test_sample-1            2.30     6.40   -                 Speech            -                 -       
  test_sample-1            7.10     9.80   -                 Music             -                 -       
  test_sample-2            2.30     3.98   -                 Music             -                 -       
  test_sample-3          

In [83]:
# Initialize evaluator with list of event labels to be evaluated and segment length
evaluator = sed_eval.sound_event.SegmentBasedMetrics(
    event_label_list=['speech', 'music'], 
    time_resolution=0.2                    # 1 second segments
)
# Loop file by file and accumulate intermediate statistics
for filename in reference_event_list.unique_files:
    evaluator.evaluate(
        reference_event_list=reference_event_list.filter(filename=filename),
        estimated_event_list=estimated_event_list.filter(filename=filename)
    ) 
metrics = evaluator.results_overall_metrics()

ValueError: ignored

## Metric values

In [70]:
log.table(
    column_headers=['Metric', 'Value'],
    cell_data=[
        [
          '<strong>F-score</strong>',
          'Precision',
          'Recall',
          '<strong>Error rate</strong>',
            'Substitutions',
            'Deletions',
            'Insertions'
        ],
        [
            metrics['f_measure']['f_measure']*100.0,
            metrics['f_measure']['precision']*100.0,
            metrics['f_measure']['recall']*100.0,
            metrics['error_rate']['error_rate'],
            metrics['error_rate']['substitution_rate'],
            metrics['error_rate']['deletion_rate'],
            metrics['error_rate']['insertion_rate'],
        ]
    ],
    row_separators=[3],
    scaling=130
)

Metric,Value
F-score,75.15
Precision,72.47
Recall,78.04
Error rate,0.41
Substitutions,0.11
Deletions,0.11
Insertions,0.19


## Class-wise metrics

In [56]:
class_metrics = evaluator.results_class_wise_metrics()
Nref = []
Nsys = []
Fscore=[]
for event_label in list(class_metrics.keys()):
    Nref.append(class_metrics[event_label]['count']['Nref'])
    Nsys.append(class_metrics[event_label]['count']['Nsys'])
    Fscore.append(class_metrics[event_label]['f_measure']['f_measure']*100.0)
log.table(
    column_headers=['Event', 'Nref', 'Nsys', 'Fscore'],
    cell_data=[
        list(class_metrics.keys()),Nref,Nsys,Fscore
    ],
    column_types=['str25', 'int', 'int', 'float2'],
    column_separators=[0,2],
    scaling=130
)

Event,Nref,Nsys,Fscore
speech,2625,2920,75.38
music,2375,2464,74.89


Output directly from `sed_eval` evaluator:

In [57]:
print(evaluator)    

Segment based metrics
  Evaluated length                  : 1543.49 sec
  Evaluated files                   : 170 
  Segment length                    : 200.00 ms

  Overall metrics (micro-average)
  F-measure
    F-measure (F1)                  : 75.15 %
    Precision                       : 72.47 %
    Recall                          : 78.04 %
  Error rate
    Error rate (ER)                 : 0.41 
    Substitution rate               : 0.11 
    Deletion rate                   : 0.11 
    Insertion rate                  : 0.19 
  Accuracy
    Sensitivity                     : 78.04 %
    Specificity                     : 85.82 %
    Balanced accuracy               : 81.93 %
    Accuracy                        : 83.31 %

  Class-wise average metrics (macro-average)
  F-measure
    F-measure (F1)                  : 75.14 %
    Precision                       : 72.56 %
    Recall                          : 77.96 %
  Error rate
    Error rate (ER)                 : 0.52 
    Deletion ra